In [1]:
import os
import re
import sys
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from Bio.PDB import *
from Bio.Data.IUPACData import protein_letters_1to3
from Bio.PDB.DSSP import DSSP
from tqdm.notebook import tqdm
from itertools import product
import warnings
warnings.filterwarnings('ignore')

In [2]:
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.edges.atomic import add_atomic_edges
from graphein.protein.graphs import construct_graph, read_pdb_to_dataframe
from graphein.protein.subgraphs import extract_subgraph_from_point
from graphein.protein.utils import save_graph_to_pdb
from graphein.molecule.edges.distance import compute_distmat, get_interacting_atoms
from graphein.protein.visualisation import plotly_protein_structure_graph

[03/27/24 01:49:11] WARNING  To use the Graphein submodule                                         ]8;id=431960;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=942583;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py#34\34]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             torch                                                                                 
                             To do so, use the following command: conda install -c pytorch torch                   

                    WARNING  To use the Graphein submodule                                         ]8;id=89574;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=264837;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/features/sequence/embeddings.py#45\45]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             biovec cannot be installed via conda                                                  
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

                    WARNING  To use the Graphein submodule graphein.protein.visualisation, you  ]8;id=676392;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=867489;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/visualisation.py#36\36]8;;\
                             need to install: pytorch3d                                                            
                             To do so, use the following command: conda install -c pytorch3d                       
                             pytorch3d                                                                             

                    WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=459994;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=587525;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/meshes.py#30\30]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: conda install -c pytorch3d pytorch3d             

In [3]:
AA_atom_pharm_dict = {
    'ALA': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',)},

    'CYS': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'SG': ('Sul',)
          },

    'ASP': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Neu',),
          'OD1': ('Acc', 'Neg'), 'OD2': ('Acc', 'Neg')
          },

    'GLU': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Hyd',), 'CD': ('Neu',),
          'OE1': ('Acc', 'Neg'), 'OE2': ('Acc', 'Neg')
          },

    'PHE': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Aro', 'Hyd'),
          'CD1': ('Aro', 'Hyd'), 'CD2': ('Aro', 'Hyd'),
          'CE1': ('Aro', 'Hyd'), 'CE2': ('Aro', 'Hyd'),
          'CZ': ('Aro', 'Hyd')
          },

    'GLY': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',)},

    'HIS': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',), 
          'ND1': ('Aro', 'Don'), 'NE2': ('Aro', 'Don'),
          'CG': ('Aro', 'Hyd'), 'CD2': ('Aro', 'Hyd'), 'CE1': ('Aro', 'Hyd')
          },

    'ILE': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG1': ('Hyd',), 'CG2': ('Hyd',), 'CD1': ('Hyd',)
          },

    'LYS': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Hyd',), 'CD': ('Hyd',), 'CE': ('Hyd',),
          'NZ': ('Don', 'Pos')
          },

    'LEU': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Hyd',), 'CD1': ('Hyd',), 'CD2': ('Hyd',)
          },

    'MET': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Hyd',), 'CE': ('Hyd',),
          'SD': ('Sul',)
          },

    'ASN': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Neu',),
          'OD1': ('Acc',),
          'ND2': ('Don',)
          },

    'PRO': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CD': ('Hyd',), 'CG': ('Hyd',)
          },

    'GLN': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Hyd',), 'CD': ('Neu',),
          'OE1': ('Acc',), 
          'NE2': ('Don',)
          },

    'ARG': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',), 
          'CG': ('Hyd',), 'CD': ('Hyd',), 'NE': ('Don',), 'CZ': ('Neu',), 
          'NH1': ('Don', 'Pos'), 'NH2': ('Don', 'Pos')
          },

    'SER': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'OG': ('Acc', 'Don')
          },

    'THR': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG2': ('Neu',),
          'OG1': ('Acc', 'Don')
          },

    'VAL': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG1': ('Hyd',), 'CG2': ('Hyd',)
          },

    'TRP': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Aro', 'Hyd'), 
          'CD1': ('Aro', 'Hyd'), 'CD2': ('Aro', 'Hyd'), 
          'CE2': ('Aro', 'Hyd'), 'CE3': ('Aro', 'Hyd'),
          'CZ2': ('Aro', 'Hyd'), 'CZ3': ('Aro', 'Hyd'),
          'CH2': ('Aro', 'Hyd'),
          'NE1': ('Don', 'Aro', 'Hyd')
          },

    'TYR': {'N': ('Don',), 'CA': ('Neu',), 'C': ('Neu',), 'O': ('Acc',), 'CB': ('Hyd',),
          'CG': ('Aro', 'Hyd'),
          'CD1': ('Aro', 'Hyd'), 'CD2': ('Aro', 'Hyd'),
          'CE1': ('Aro', 'Hyd'), 'CE2': ('Aro', 'Hyd'),
          'CZ': ('Aro', 'Hyd'),
          'OH': ('Acc', 'Don')
          },
}

In [4]:
AA_vector_pharm_dict = {
 'A': np.array([1, 0, 0, 1, 1, 0, 0, 2]),
 'C': np.array([1, 0, 0, 1, 1, 0, 1, 2]),
 'D': np.array([1, 0, 2, 3, 1, 0, 0, 3]),
 'E': np.array([2, 0, 2, 3, 1, 0, 0, 3]),
 'F': np.array([7, 0, 0, 1, 1, 6, 0, 2]),
 'G': np.array([0, 0, 0, 1, 1, 0, 0, 2]),
 'H': np.array([4, 0, 0, 1, 3, 5, 0, 2]),
 'I': np.array([4, 0, 0, 1, 1, 0, 0, 2]),
 'K': np.array([4, 1, 0, 1, 2, 0, 0, 2]),
 'L': np.array([4, 0, 0, 1, 1, 0, 0, 2]),
 'M': np.array([3, 0, 0, 1, 1, 0, 1, 2]),
 'N': np.array([1, 0, 0, 2, 2, 0, 0, 3]),
 'P': np.array([3, 0, 0, 1, 1, 0, 0, 2]),
 'Q': np.array([2, 0, 0, 2, 2, 0, 0, 3]),
 'R': np.array([3, 2, 0, 1, 4, 0, 0, 3]),
 'S': np.array([1, 0, 0, 2, 2, 0, 0, 2]),
 'T': np.array([1, 0, 0, 2, 2, 0, 0, 3]),
 'V': np.array([3, 0, 0, 1, 1, 0, 0, 2]),
 'W': np.array([10,0, 0, 1, 2, 9, 0, 2]),
 'Y': np.array([7, 0, 0, 2, 2, 6, 0, 2])
 }

In [5]:
def extractResidueEnvironment(pdb_id, mutation, aa, num_aa, chain) -> pd.DataFrame:
    """
    Извлекает окружение мутирующего остатка, создавая подграф вокруг этого остатка.
    Подграф сохраняется в формате PDB и затем преобразуется в DataFrame для дальнейшего анализа.
    DataFrame скачивается в формате pickle

    - pdb_id (str): Идентификатор PDB структуры белка.
    - mutation (str): Мутация в формате 'AAnumAA', где AA - аминокислота, num - номер остатка.
    - chain (str): Цепь, к которой относится мутирующий остаток.

    Пример использования:
    >>> residue_environment = extractResidueEnvironment('1A2K', 'F149A', 'A')
    """

    aa = protein_letters_1to3[aa].upper() # F –> Phe –> PHE


    # Делаем граф из всего белка, если еще не делали на предыдущем шаге
    if pdb_id not in PDB_graphs_dict.keys():
        params_to_change = {"granularity": "atom", "edge_construction_functions": [add_atomic_edges]}
        config = ProteinGraphConfig(**params_to_change)
        graph = construct_graph(config=config, pdb_code=pdb_id)
        # Сохраняем граф в словаре
        PDB_graphs_dict[pdb_id] = graph
    else:
        # Если граф уже существует, используем его
        graph = PDB_graphs_dict[pdb_id]
    
    # Поиск координат СА мутируемого остатка
    for node, data in graph.nodes(data=True):
        if f'{chain}:{aa}:{num_aa}:CA' in node:
            mut_center = data['coords']
    

    # Выделение подграфа residue_environment_sg вокруг мутируемого остатка
    residue_environment_sg = extract_subgraph_from_point(graph, centre_point=(mut_center), radius=residue_environment_radius)


    # Сохранение подграфа residue_environment_sg в pdb формате
    save_graph_to_pdb(residue_environment_sg, f'./cutPDBs/{pdb_id}_{mutation}_cut.pdb')

    # Перевод pdb в датафрейм
    residue_environment = read_pdb_to_dataframe(f'./cutPDBs/{pdb_id}_{mutation}_cut.pdb')

    return residue_environment

In [6]:
def extractEnvironmentPharms(residue_environment):

    residue_environment['Pharmacophore'] = ''

    def add_pharm_name(row):
        residue_name = row['residue_name']
        atom_name = row['atom_name']
        atom_pharm = AA_atom_pharm_dict[residue_name][atom_name]
        # print(residue_name, atom_name, 'Фармакофор найден в словаре!')
        return atom_pharm

    # Применение функции к каждой строке DataFrame для заполнения колонки 'Pharmacophore'
    residue_environment['Pharmacophore'] = residue_environment.apply(add_pharm_name, axis=1)

    # Сохранение датафрейма в файл pkl
    # os.makedirs('./res_env_pkl', exist_ok=True)
    # residue_environment.to_pickle(f'./res_env_pkl/{pdb_id}_{mutation}_env.pkl')

    return residue_environment

In [7]:
# Проверка, как выглядит residue_environment_sg
# plotly_protein_structure_graph(residue_environment_sg, node_size_min=4, node_size_multiplier=2)

In [8]:
def calculateAtomicPairwiseDist(res_env) -> pd.DataFrame:

    coords = res_env.filter(like='_coord')
    distMatrix = compute_distmat(coords)

    pharmacophores = list(res_env['Pharmacophore'])
                          
    # Преобразование списка pharmacophores в строковый тип данных, это необходимо чтобы в getFrequency искать класс фармакофора в индексах (одному атому соответствуют несколько)
    pharmacophores_str = pd.Series(pharmacophores).astype(str)

    # Создание DataFrame с индексами, означающими фармакофор, в строковом типе данных
    distMatrix = pd.DataFrame(distMatrix, index=pharmacophores_str, 
                                        columns=pharmacophores_str)

    return distMatrix


In [9]:
def getFrequency(distMatrix, dist, classes):

    cls1, cls2 = classes

    distMatrix.index = distMatrix.index.astype(str)
    distMatrix.columns = distMatrix.columns.astype(str)

    # Проверка наличия фармакофоров в окружении АК, если какого-то класса нет — возвращаем ноль
    if not any(distMatrix.index.str.contains(cls1)) or not any(distMatrix.columns.str.contains(cls2)):
        return 0
        
    # Выбираем только значения на пересечении классов
    # Используем метод str.contains для фильтрации строк и столбцов
    filt_distMatrix = distMatrix.loc[distMatrix.index.str.contains(cls1), distMatrix.columns.str.contains(cls2)]
    
    # дополнительно проверяем > 0.001
    frequency_matrix = ((filt_distMatrix > 0.001) & (filt_distMatrix > (dist - Dstep)) & (filt_distMatrix < dist))
    
    # sum().sum() — первый раз по строкам, второй раз суммы строк
    frequency = int(frequency_matrix.sum().sum())
    
    # если считаем для атомов одного класса, частоту делим пополам, т.к. dist(A1,A2) == dist(A2,А1)
    if cls1 == cls2:
        frequency /= 2

    return frequency

In [10]:
def useDSSP(pdb_id, num_aa, chain):
    p = PDBParser(QUIET=True)
    structure = p.get_structure('d', f'./PDBs/{pdb_id}.pdb')
    model = structure[0]

    # применяю модуль DSSP к pdb файлу
    dssp = DSSP(model, f'./PDBs/{pdb_id}.pdb', dssp='mkdssp', acc_array='Sander', file_type='PDB')

    # выбираю из выдачи модуля информацию о целевой АК
    info_about_aa = dssp[(chain, (' ', int(num_aa), ' '))]
    secondary_structure = info_about_aa[2]
    rsa = info_about_aa[3]
    
    
    return rsa, secondary_structure

In [11]:
# для некоторых АК RSA == 0, такое выдает DSSP
useDSSP('1FC2', 135, 'C')

(0.0, 'H')

In [12]:
# немного измененная функция из статьи 10.1109/TCBB.2022.3233627 
def residue_feature(AA) -> np.array:
    def AAcharge(AA):
        if AA in ['D','E']:
            return -1.
        elif AA in ['R','H','K']:
            return 1.
        else:
            return 0.

    residueFeature = []

    AAvolume = {'A':88.6, 'R':173.4, 'D':111.1, 'N':114.1, 'C':108.5, 'E':138.4, 'Q':143.8, 'G':60.1, 'H':153.2, 'I':166.7, 'L':166.7, 'K':168.6, 'M':162.9, 'F':189.9, 'P':112.7, 'S':89., 'T':116.1, 'W':227.8, 'Y':193.6, 'V':140. }
    AAhydropathy = {'A':1.8, 'R':-4.5, 'N':-3.5, 'D': -3.5, 'C': 2.5, 'E':-3.5, 'Q':-3.5, 'G':-0.4, 'H':-3.2, 'I':4.5, 'L':3.8, 'K':-3.9, 'M':1.9, 'F':2.8, 'P':-1.6, 'S':-0.8, 'T':-0.7, 'W':-0.9, 'Y':-1.3, 'V':4.2}
    AAarea = {'A':115.,'R':225.,'D':150.,'N':160.,'C':135.,'E':190.,'Q':180.,'G':75.,'H':195.,'I':175.,'L':170.,'K':200.,'M':185.,'F':210.,'P':145.,'S':115.,'T':140.,'W':255.,'Y':230.,'V':155.}
    AAweight = {'A':89.094,'R':174.203,'N':132.119,'D':133.104,'C':121.154,'E':147.131,'Q':146.146,'G':75.067,'H':155.156,'I':131.175,'L':131.175,'K':146.189,'M':149.208,'F':165.192,'P':115.132,'S':105.093,'T':119.12,'W':204.228,'Y':181.191,'V':117.148}
    AAflexibily = {'A':'1','R':'81','N':'36','D':'18','C':'3','E':'54','Q':'108','G':'1','H':'36','I':'9','L':'9','K':'81','M':'27','F':'18','P':'2','S':'3','T':'3','W':'36','Y':'18','V':'3'}
    
    residueFeature.append(AAvolume[AA])
    residueFeature.append(AAhydropathy[AA])
    residueFeature.append(AAarea[AA])
    residueFeature.append(AAweight[AA])
    residueFeature.append(AAcharge(AA))
    residueFeature.append(AAflexibily[AA])

    if AA in ('A','G','I','L','P','V'):chemical = 0
    elif AA in ('R','H','K'):chemical = 1
    elif AA in ('D','E'):chemical = 2
    elif AA in ('N','Q'):chemical = 3
    elif AA in ('C','M'):chemical = 4
    elif AA in ('S','T'):chemical = 5
    elif AA in ('F','W','Y'):chemical = 6
    residueFeature.append(chemical)

    if AA in ('G','A','S'):size = 0
    elif AA in ('C','D','P','N','T'):size = 1
    elif AA in ('E','V','Q','H'):size = 2
    elif AA in ('M','I','L','K','R'):size = 3
    elif AA in ('F','Y','W'):size = 4
    residueFeature.append(size)

    if AA in ('R','W','K'):hbonds = 0
    if AA in ('A','C','G','I','L','M','F','P','V'):hbonds = 1 
    if AA in ('N','Q','S','T','H','Y'):hbonds = 3  
    if AA in ('D','E'):hbonds = 4
    residueFeature.append(hbonds)
    
    return np.array(residueFeature).astype(float)

In [20]:
MutationSet = pd.read_csv('../datasets/mCSM-AB2_dataset_short.csv')

AtomClass = ('Hyd','Pos','Neg','Acc','Don','Aro','Sul','Neu')


residue_environment_radius = 10

# Будем считать кол-во контактов по сферическим слоям толщиной Dstep
Dstep = 4
Dmin = Dstep
Dmax = residue_environment_radius * 2 + 1  # Dmax включительно —> + 1

PDB_graphs_dict = {}

In [14]:
def Generate_CSM(pdb_id, mutation, aa, num_aa, chain):
            
    residue_environment = extractResidueEnvironment(pdb_id, mutation, aa, num_aa, chain)
    residue_environment = extractEnvironmentPharms(residue_environment)
    distMatrix = calculateAtomicPairwiseDist(residue_environment)
    
    
    CSM_row = []
    for dist in range(Dmin, Dmax, Dstep):
        print(f'Distance == {dist}')
        for classes in product(AtomClass, repeat=2): # берем комбинацию из 2 классов с повторениями
            frequency = getFrequency(distMatrix, dist, classes)
            CSM_row.append(frequency)
            print(classes, 'frequency =', frequency)

    return CSM_row

In [15]:
# CSM_row = Generate_CSM('1FC2', 'F149A', 'F', 149, 'C')
# pd.set_option('display.max_rows', 5)
# pd.set_option('display.max_columns', 5)
# CSM_row

In [22]:
def GenerateFeatures(MutationSet):
    Features = []
    row_names = []
    num_rows = len(MutationSet)

    with tqdm(total=num_rows, desc="Processing rows") as progressbar:
        for i, (index, Mutation) in enumerate(MutationSet.iterrows()):

            pdb_id = Mutation['PDB']
            mutation = Mutation['mutation']
            chain = Mutation['chain']
            ddG = Mutation['Exp. DDG']
            aa, num_aa, mut_aa = re.match(r"([A-Za-z])(\d+)([A-Za-z])", mutation).groups() #F149A –> F+149+A
            
            print(pdb_id, mutation)


            FeaturesRow = []

            try:
                CSM_row = Generate_CSM(pdb_id, mutation, aa, num_aa, chain)
            except Exception as e:
                print(f'CSM неудался для {pdb_id}_{mutation}')
                print('Ошибка: ')
                print(f'{e}')
                continue
            row_names.append(f'{pdb_id}_{mutation}')
            pharmacophore_count = AA_vector_pharm_dict[mut_aa] - AA_vector_pharm_dict[aa]
            # arpeggio_contacts = Arpeggio(pdb_id, mutation, chain) 
            rsa, secondary_structure = useDSSP(pdb_id, num_aa, chain)
            residueFeature = residue_feature(mut_aa) - residue_feature(aa)
            
            
            FeaturesRow.extend(CSM_row)
            FeaturesRow.extend(pharmacophore_count) 
            # FeaturesRow.extend(arpeggio_contacts) # межатомные контакты
            FeaturesRow.extend(residueFeature) # бх признаки из статьи 
            FeaturesRow.extend([rsa, secondary_structure])
            FeaturesRow.append(ddG)


            Features.append(FeaturesRow)
            
            
            progressbar.update()


    # Создаем список формата PDB_Mutation для подписи строк в датафрейме
    # row_names = MutationSet.apply(lambda row: f"{row['PDB']}_{row['mutation']}", axis=1).tolist()

    # Создаем список названий фичей для подписи колонок в датафрейме
    combinations = list(product(AtomClass, repeat=2))
    column_names = [f"{cls1}_{cls2}_{dist}" for dist in range(Dmin, Dmax, Dstep) for cls1, cls2 in combinations]
    column_names.extend(['∆Hyd', '∆Pos', '∆Neg', '∆Acc', '∆Don', '∆Aro', '∆Sul', '∆Neu'])
    column_names.extend(['∆AAvolume', '∆AAhydropathy', '∆AAarea', '∆AAweight', '∆AAcharge', '∆AAflexibily', '∆AAchemical', '∆AAsize', '∆AAhbonds'])
    column_names.extend(['RSA', 'Secondary_structure'])
    column_names.append('ddG')
    Features = pd.DataFrame(Features, index=row_names, columns=column_names)

    return Features

## Ниже временный и тестовый код

In [23]:
Features = GenerateFeatures(MutationSet)
Features

Processing rows:   0%|          | 0/25 [00:00<?, ?it/s]

Output()

1AHW D178A


[03/27/24 01:51:35] DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=870939;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=355350;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/graphs.py#188\188]8;;\

                    DEBUG    Detected 9830 total nodes                                                ]8;id=453832;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=548278;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/graphs.py#435\435]8;;\

...

[03/27/24 01:53:32] DEBUG    Found 124 nodes in the spatial point-radius subgraph.                 ]8;id=749328;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=125529;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/subgraphs.py#121\121]8;;\

                    DEBUG    Creating subgraph from nodes: ['H:ASP:56:O', 'H:ILE:51:O',             ]8;id=874976;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=126523;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/subgraphs.py#58\58]8;;\
                             'H:TRP:52:CA', 'H:LYS:57:CB', 'H:ASP:56:CB', 'H:TRP:52:CD1',                          
                             'H:MET:34:CA', 'H:LYS:57:NZ', 'H:ASP:56:CG', 'I:LYS:52:CE',                           
                             'I:VAL:51:CG1', 'H:TRP:53:CD1', 'H:THR:73:CB', 'H:ILE:51:CD1',                        
                             'H:TRP:52:CE3', 'H:TRP:52:C', 'H:TRP:53:CD2', 'H:TRP:53:CB',                          
                             'H:THR:30:O', 'H:LEU:29:CG', 'H:LYS:71:CB', 'H:LEU:29:CD1',                           
                             'H:TRP:53:CZ2', 'H:LYS:57:C', 'H:ILE:51:N', 'H:ILE:51:C',                             
                             'H:TYR:32:O', 'H:ILE:51:CA', 'H:TRP:53:CE3', 'H:THR:30:OG1',                          
                             'H:ILE:51:CB', 'H:THR:31:CA', 'H:THR:31:C', 'H:ILE:51:CG2',                           
                             'H:TRP:53:N', 'H:TYR:32:CD2', 'H:LEU:29:N', 'H:THR:30:CG2',                           
                             'H:TYR:32:N', 'H:TRP:52:CG', 'H:TRP:53:O', 'H:TRP:52:CD2',                            
                             'H:TRP:52:CB', 'H:TYR:32:CG', 'H:LYS:57:CD', 'H:LEU:29:O',                            
                             'I:LYS:52:NZ', 'H:GLY:33:C', 'H:ASP:54:CA', 'H:ASP:55:OD2',                           
                             'H:MET:34:O', 'H:GLY:35:N', 'H:TYR:32:CZ', 'H:LEU:29:C', 'H:TYR:32:C',                
                             'H:TRP:52:CH2', 'H:ILE:51:CG1', 'H:THR:31:O', 'H:TRP:52:CZ2',                         
                             'H:TRP:52:CZ3', 'H:GLY:33:O', 'H:ASP:56:OD1', 'H:LYS:71:N',                           
                             'H:ASP:55:N', 'H:TYR:32:CD1', 'H:MET:34:C', 'H:LYS:57:N',                             
                             'H:LYS:57:CG', 'H:LYS:71:CG', 'H:GLY:33:N', 'H:ASP:55:CG',                            
                             'H:ASP:56:N', 'H:TYR:32:CA', 'H:TRP:52:NE1', 'H:THR:73:OG1',                          
                             'I:ASP:76:OD2', 'H:ASP:56:CA', 'H:THR:30:C', 'H:LYS:71:O',                            
                             'H:ASP:56:OD2', 'H:LYS:57:CE', 'H:LEU:29:CD2', 'H:LYS:57:CA',                         
                             'H:LEU:29:CB', 'H:THR:30:N', 'H:TRP:53:CG', 'H:ASP:54:OD1',                           
                             'H:ASP:55:O', 'H:ASP:55:C', 'H:LYS:71:CD', 'H:TRP:53:CA',                             
                             'H:THR:30:CB', 'H:MET:34:CB', 'H:TYR:32:CB', 'H:THR:31:N',                            
                             'H:ASP:54:O', 'H:ASP:56:C', 'H:ASP:54:OD2', 'I:LYS:52:CD',                            
                             'H:THR:30:CA', 'H:GLY:35:CA', 'H:TRP:53:C', 'H:LYS:71:NZ',                            
                             'H:TYR:32:CE1', 'H:TRP:52:O', 'H:ASP:54:CB', 'H:ASP:55:CA',                           
                             'H:TRP:53:NE1', 'H:ASP:54:CG', 'H:LEU:29:CA', 'H:TRP:52:N',                           
                             'H:GLY:33:CA', 'H:TRP:52:CE2', 'H:LYS:71:CE', 'I:LYS:52:CG',                          
                             'H:TRP:53:CZ3', 'H:ASP:54:N', 'H:LYS:71:CA', 'H:ASP:54:C',                            
                             'H:MET:34:N', 'H:ASP:55:OD1', 'H:TRP:53:CH2', 'H:ASP:55:CB',                          
                             'H:TRP:53:CE2'].                                                                      

                    INFO     Successfully saved graph to ./cutPDBs/1JRH_D54A_cut.pdb                   ]8;id=50677;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/utils.py\utils.py]8;;\:]8;id=559904;file:///Users/chekalin/miniforge3/lib/python3.10/site-packages/graphein/protein/utils.py#415\415]8;;\

Distance == 4
('Hyd', 'Hyd') frequency = 138.0
('Hyd', 'Pos') frequency = 11
('Hyd', 'Neg') frequency = 16
('Hyd', 'Acc') frequency = 62
('Hyd', 'Don') frequency = 85
('Hyd', 'Aro') frequency = 187
('Hyd', 'Sul') frequency = 0
('Hyd', 'Neu') frequency = 83
('Pos', 'Hyd') frequency = 11
('Pos', 'Pos') frequency = 0.0
('Pos', 'Neg') frequency = 5
('Pos', 'Acc') frequency = 7
('Pos', 'Don') frequency = 1
('Pos', 'Aro') frequency = 0
('Pos', 'Sul') frequency = 0
('Pos', 'Neu') frequency = 3
('Neg', 'Hyd') frequency = 16
('Neg', 'Pos') frequency = 5
('Neg', 'Neg') frequency = 3.0
('Neg', 'Acc') frequency = 10
('Neg', 'Don') frequency = 9
('Neg', 'Aro') frequency = 0
('Neg', 'Sul') frequency = 0
('Neg', 'Neu') frequency = 17
('Acc', 'Hyd') frequency = 62
('Acc', 'Pos') frequency = 7
('Acc', 'Neg') frequency = 10
('Acc', 'Acc') frequency = 15.0
('Acc', 'Don') frequency = 52
('Acc', 'Aro') frequency = 12
('Acc', 'Sul') frequency = 0
('Acc', 'Neu') frequency = 88
('Don', 'Hyd') frequency = 85
(

,Hyd_Hyd_4,Hyd_Pos_4,Hyd_Neg_4,Hyd_Acc_4,Hyd_Don_4,Hyd_Aro_4,Hyd_Sul_4,Hyd_Neu_4,Pos_Hyd_4,Pos_Pos_4,Pos_Neg_4,Pos_Acc_4,Pos_Don_4,Pos_Aro_4,Pos_Sul_4,Pos_Neu_4,Neg_Hyd_4,Neg_Pos_4,Neg_Neg_4,Neg_Acc_4,Neg_Don_4,Neg_Aro_4,Neg_Sul_4,Neg_Neu_4,Acc_Hyd_4,Acc_Pos_4,Acc_Neg_4,Acc_Acc_4,Acc_Don_4,Acc_Aro_4,Acc_Sul_4,Acc_Neu_4,Don_Hyd_4,Don_Pos_4,Don_Neg_4,Don_Acc_4,Don_Don_4,Don_Aro_4,Don_Sul_4,Don_Neu_4,Aro_Hyd_4,Aro_Pos_4,Aro_Neg_4,Aro_Acc_4,Aro_Don_4,Aro_Aro_4,Aro_Sul_4,Aro_Neu_4,Sul_Hyd_4,Sul_Pos_4,Sul_Neg_4,Sul_Acc_4,Sul_Don_4,Sul_Aro_4,Sul_Sul_4,Sul_Neu_4,Neu_Hyd_4,Neu_Pos_4,Neu_Neg_4,Neu_Acc_4,Neu_Don_4,Neu_Aro_4,Neu_Sul_4,Neu_Neu_4,Hyd_Hyd_8,Hyd_Pos_8,Hyd_Neg_8,Hyd_Acc_8,Hyd_Don_8,Hyd_Aro_8,Hyd_Sul_8,Hyd_Neu_8,Pos_Hyd_8,Pos_Pos_8,Pos_Neg_8,Pos_Acc_8,Pos_Don_8,Pos_Aro_8,Pos_Sul_8,Pos_Neu_8,Neg_Hyd_8,Neg_Pos_8,Neg_Neg_8,Neg_Acc_8,Neg_Don_8,Neg_Aro_8,Neg_Sul_8,Neg_Neu_8,Acc_Hyd_8,Acc_Pos_8,Acc_Neg_8,Acc_Acc_8,Acc_Don_8,Acc_Aro_8,Acc_Sul_8,Acc_Neu_8,Don_Hyd_8,Don_Pos_8,Don_Neg_8,Don_Acc_8,Don_Don_8,Don_Aro_8,Don_Sul_8,Don_Neu_8,Aro_Hyd_8,Aro_Pos_8,Aro_Neg_8,Aro_Acc_8,Aro_Don_8,Aro_Aro_8,Aro_Sul_8,Aro_Neu_8,Sul_Hyd_8,Sul_Pos_8,Sul_Neg_8,Sul_Acc_8,Sul_Don_8,Sul_Aro_8,Sul_Sul_8,Sul_Neu_8,Neu_Hyd_8,Neu_Pos_8,Neu_Neg_8,Neu_Acc_8,Neu_Don_8,Neu_Aro_8,Neu_Sul_8,Neu_Neu_8,Hyd_Hyd_12,Hyd_Pos_12,Hyd_Neg_12,Hyd_Acc_12,Hyd_Don_12,Hyd_Aro_12,Hyd_Sul_12,Hyd_Neu_12,Pos_Hyd_12,Pos_Pos_12,Pos_Neg_12,Pos_Acc_12,Pos_Don_12,Pos_Aro_12,Pos_Sul_12,Pos_Neu_12,Neg_Hyd_12,Neg_Pos_12,Neg_Neg_12,Neg_Acc_12,Neg_Don_12,Neg_Aro_12,Neg_Sul_12,Neg_Neu_12,Acc_Hyd_12,Acc_Pos_12,Acc_Neg_12,Acc_Acc_12,Acc_Don_12,Acc_Aro_12,Acc_Sul_12,Acc_Neu_12,Don_Hyd_12,Don_Pos_12,Don_Neg_12,Don_Acc_12,Don_Don_12,Don_Aro_12,Don_Sul_12,Don_Neu_12,Aro_Hyd_12,Aro_Pos_12,Aro_Neg_12,Aro_Acc_12,Aro_Don_12,Aro_Aro_12,Aro_Sul_12,Aro_Neu_12,Sul_Hyd_12,Sul_Pos_12,Sul_Neg_12,Sul_Acc_12,Sul_Don_12,Sul_Aro_12,Sul_Sul_12,Sul_Neu_12,Neu_Hyd_12,Neu_Pos_12,Neu_Neg_12,Neu_Acc_12,Neu_Don_12,Neu_Aro_12,Neu_Sul_12,Neu_Neu_12,Hyd_Hyd_16,Hyd_Pos_16,Hyd_Neg_16,Hyd_Acc_16,Hyd_Don_16,Hyd_Aro_16,Hyd_Sul_16,Hyd_Neu_16,Pos_Hyd_16,Pos_Pos_16,Pos_Neg_16,Pos_Acc_16,Pos_Don_16,Pos_Aro_16,Pos_Sul_16,Pos_Neu_16,Neg_Hyd_16,Neg_Pos_16,Neg_Neg_16,Neg_Acc_16,Neg_Don_16,Neg_Aro_16,Neg_Sul_16,Neg_Neu_16,Acc_Hyd_16,Acc_Pos_16,Acc_Neg_16,Acc_Acc_16,Acc_Don_16,Acc_Aro_16,Acc_Sul_16,Acc_Neu_16,Don_Hyd_16,Don_Pos_16,Don_Neg_16,Don_Acc_16,Don_Don_16,Don_Aro_16,Don_Sul_16,Don_Neu_16,Aro_Hyd_16,Aro_Pos_16,Aro_Neg_16,Aro_Acc_16,Aro_Don_16,Aro_Aro_16,Aro_Sul_16,Aro_Neu_16,Sul_Hyd_16,Sul_Pos_16,Sul_Neg_16,Sul_Acc_16,Sul_Don_16,Sul_Aro_16,Sul_Sul_16,Sul_Neu_16,Neu_Hyd_16,Neu_Pos_16,Neu_Neg_16,Neu_Acc_16,Neu_Don_16,Neu_Aro_16,Neu_Sul_16,Neu_Neu_16,Hyd_Hyd_20,Hyd_Pos_20,Hyd_Neg_20,Hyd_Acc_20,Hyd_Don_20,Hyd_Aro_20,Hyd_Sul_20,Hyd_Neu_20,Pos_Hyd_20,Pos_Pos_20,Pos_Neg_20,Pos_Acc_20,Pos_Don_20,Pos_Aro_20,Pos_Sul_20,Pos_Neu_20,Neg_Hyd_20,Neg_Pos_20,Neg_Neg_20,Neg_Acc_20,Neg_Don_20,Neg_Aro_20,Neg_Sul_20,Neg_Neu_20,Acc_Hyd_20,Acc_Pos_20,Acc_Neg_20,Acc_Acc_20,Acc_Don_20,Acc_Aro_20,Acc_Sul_20,Acc_Neu_20,Don_Hyd_20,Don_Pos_20,Don_Neg_20,Don_Acc_20,Don_Don_20,Don_Aro_20,Don_Sul_20,Don_Neu_20,Aro_Hyd_20,Aro_Pos_20,Aro_Neg_20,Aro_Acc_20,Aro_Don_20,Aro_Aro_20,Aro_Sul_20,Aro_Neu_20,Sul_Hyd_20,Sul_Pos_20,Sul_Neg_20,Sul_Acc_20,Sul_Don_20,Sul_Aro_20,Sul_Sul_20,Sul_Neu_20,Neu_Hyd_20,Neu_Pos_20,Neu_Neg_20,Neu_Acc_20,Neu_Don_20,Neu_Aro_20,Neu_Sul_20,Neu_Neu_20,∆Hyd,∆Pos,∆Neg,∆Acc,∆Don,∆Aro,∆Sul,∆Neu,∆AAvolume,∆AAhydropathy,∆AAarea,∆AAweight,∆AAcharge,∆AAflexibily,∆AAchemical,∆AAsize,∆AAhbonds,RSA,Secondary_structure,ddG
1AHW_D178A,91.0,7,13,60,60,85,1,76,7,0.0,2,3,1,0,0,4,13,2,4.0,11,8,0,0,14,60,3,11,13.0,45,10,1,84,60,1,8,45,14.0,12,0,70,85,0,0,10,12,31.0,0,0,1,0,0,1,0,0,0.0,0,76,4,14,84,70,0,0,57.0,428.0,23,71,326,297,266,16,513,23,0.0,7,20,14,8,0,19,71,7,7.0,47,39,17,0,60,326,20,47,85.0,137,68,3,234,297,14,39,137,58.0,53,3,225,266,8,17,68,53,52.0,6,83,16,0,0,3,3,6,0.0,6,513,19,60,234,225,83,6,190.0,482.0,54,186,495,415,209,22,699,54,1.0,6,16,13,18,0,25,

In [24]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
Features

,Hyd_Hyd_4,Hyd_Pos_4,Hyd_Neg_4,Hyd_Acc_4,Hyd_Don_4,Hyd_Aro_4,Hyd_Sul_4,Hyd_Neu_4,Pos_Hyd_4,Pos_Pos_4,Pos_Neg_4,Pos_Acc_4,Pos_Don_4,Pos_Aro_4,Pos_Sul_4,Pos_Neu_4,Neg_Hyd_4,Neg_Pos_4,Neg_Neg_4,Neg_Acc_4,Neg_Don_4,Neg_Aro_4,Neg_Sul_4,Neg_Neu_4,Acc_Hyd_4,Acc_Pos_4,Acc_Neg_4,Acc_Acc_4,Acc_Don_4,Acc_Aro_4,Acc_Sul_4,Acc_Neu_4,Don_Hyd_4,Don_Pos_4,Don_Neg_4,Don_Acc_4,Don_Don_4,Don_Aro_4,Don_Sul_4,Don_Neu_4,Aro_Hyd_4,Aro_Pos_4,Aro_Neg_4,Aro_Acc_4,Aro_Don_4,Aro_Aro_4,Aro_Sul_4,Aro_Neu_4,Sul_Hyd_4,Sul_Pos_4,Sul_Neg_4,Sul_Acc_4,Sul_Don_4,Sul_Aro_4,Sul_Sul_4,Sul_Neu_4,Neu_Hyd_4,Neu_Pos_4,Neu_Neg_4,Neu_Acc_4,Neu_Don_4,Neu_Aro_4,Neu_Sul_4,Neu_Neu_4,Hyd_Hyd_8,Hyd_Pos_8,Hyd_Neg_8,Hyd_Acc_8,Hyd_Don_8,Hyd_Aro_8,Hyd_Sul_8,Hyd_Neu_8,Pos_Hyd_8,Pos_Pos_8,Pos_Neg_8,Pos_Acc_8,Pos_Don_8,Pos_Aro_8,Pos_Sul_8,Pos_Neu_8,Neg_Hyd_8,Neg_Pos_8,Neg_Neg_8,Neg_Acc_8,Neg_Don_8,Neg_Aro_8,Neg_Sul_8,Neg_Neu_8,Acc_Hyd_8,Acc_Pos_8,Acc_Neg_8,Acc_Acc_8,Acc_Don_8,Acc_Aro_8,Acc_Sul_8,Acc_Neu_8,Don_Hyd_8,Don_Pos_8,Don_Neg_8,Don_Acc_8,Don_Don_8,Don_Aro_8,Don_Sul_8,Don_Neu_8,Aro_Hyd_8,Aro_Pos_8,Aro_Neg_8,Aro_Acc_8,Aro_Don_8,Aro_Aro_8,Aro_Sul_8,Aro_Neu_8,Sul_Hyd_8,Sul_Pos_8,Sul_Neg_8,Sul_Acc_8,Sul_Don_8,Sul_Aro_8,Sul_Sul_8,Sul_Neu_8,Neu_Hyd_8,Neu_Pos_8,Neu_Neg_8,Neu_Acc_8,Neu_Don_8,Neu_Aro_8,Neu_Sul_8,Neu_Neu_8,Hyd_Hyd_12,Hyd_Pos_12,Hyd_Neg_12,Hyd_Acc_12,Hyd_Don_12,Hyd_Aro_12,Hyd_Sul_12,Hyd_Neu_12,Pos_Hyd_12,Pos_Pos_12,Pos_Neg_12,Pos_Acc_12,Pos_Don_12,Pos_Aro_12,Pos_Sul_12,Pos_Neu_12,Neg_Hyd_12,Neg_Pos_12,Neg_Neg_12,Neg_Acc_12,Neg_Don_12,Neg_Aro_12,Neg_Sul_12,Neg_Neu_12,Acc_Hyd_12,Acc_Pos_12,Acc_Neg_12,Acc_Acc_12,Acc_Don_12,Acc_Aro_12,Acc_Sul_12,Acc_Neu_12,Don_Hyd_12,Don_Pos_12,Don_Neg_12,Don_Acc_12,Don_Don_12,Don_Aro_12,Don_Sul_12,Don_Neu_12,Aro_Hyd_12,Aro_Pos_12,Aro_Neg_12,Aro_Acc_12,Aro_Don_12,Aro_Aro_12,Aro_Sul_12,Aro_Neu_12,Sul_Hyd_12,Sul_Pos_12,Sul_Neg_12,Sul_Acc_12,Sul_Don_12,Sul_Aro_12,Sul_Sul_12,Sul_Neu_12,Neu_Hyd_12,Neu_Pos_12,Neu_Neg_12,Neu_Acc_12,Neu_Don_12,Neu_Aro_12,Neu_Sul_12,Neu_Neu_12,Hyd_Hyd_16,Hyd_Pos_16,Hyd_Neg_16,Hyd_Acc_16,Hyd_Don_16,Hyd_Aro_16,Hyd_Sul_16,Hyd_Neu_16,Pos_Hyd_16,Pos_Pos_16,Pos_Neg_16,Pos_Acc_16,Pos_Don_16,Pos_Aro_16,Pos_Sul_16,Pos_Neu_16,Neg_Hyd_16,Neg_Pos_16,Neg_Neg_16,Neg_Acc_16,Neg_Don_16,Neg_Aro_16,Neg_Sul_16,Neg_Neu_16,Acc_Hyd_16,Acc_Pos_16,Acc_Neg_16,Acc_Acc_16,Acc_Don_16,Acc_Aro_16,Acc_Sul_16,Acc_Neu_16,Don_Hyd_16,Don_Pos_16,Don_Neg_16,Don_Acc_16,Don_Don_16,Don_Aro_16,Don_Sul_16,Don_Neu_16,Aro_Hyd_16,Aro_Pos_16,Aro_Neg_16,Aro_Acc_16,Aro_Don_16,Aro_Aro_16,Aro_Sul_16,Aro_Neu_16,Sul_Hyd_16,Sul_Pos_16,Sul_Neg_16,Sul_Acc_16,Sul_Don_16,Sul_Aro_16,Sul_Sul_16,Sul_Neu_16,Neu_Hyd_16,Neu_Pos_16,Neu_Neg_16,Neu_Acc_16,Neu_Don_16,Neu_Aro_16,Neu_Sul_16,Neu_Neu_16,Hyd_Hyd_20,Hyd_Pos_20,Hyd_Neg_20,Hyd_Acc_20,Hyd_Don_20,Hyd_Aro_20,Hyd_Sul_20,Hyd_Neu_20,Pos_Hyd_20,Pos_Pos_20,Pos_Neg_20,Pos_Acc_20,Pos_Don_20,Pos_Aro_20,Pos_Sul_20,Pos_Neu_20,Neg_Hyd_20,Neg_Pos_20,Neg_Neg_20,Neg_Acc_20,Neg_Don_20,Neg_Aro_20,Neg_Sul_20,Neg_Neu_20,Acc_Hyd_20,Acc_Pos_20,Acc_Neg_20,Acc_Acc_20,Acc_Don_20,Acc_Aro_20,Acc_Sul_20,Acc_Neu_20,Don_Hyd_20,Don_Pos_20,Don_Neg_20,Don_Acc_20,Don_Don_20,Don_Aro_20,Don_Sul_20,Don_Neu_20,Aro_Hyd_20,Aro_Pos_20,Aro_Neg_20,Aro_Acc_20,Aro_Don_20,Aro_Aro_20,Aro_Sul_20,Aro_Neu_20,Sul_Hyd_20,Sul_Pos_20,Sul_Neg_20,Sul_Acc_20,Sul_Don_20,Sul_Aro_20,Sul_Sul_20,Sul_Neu_20,Neu_Hyd_20,Neu_Pos_20,Neu_Neg_20,Neu_Acc_20,Neu_Don_20,Neu_Aro_20,Neu_Sul_20,Neu_Neu_20,∆Hyd,∆Pos,∆Neg,∆Acc,∆Don,∆Aro,∆Sul,∆Neu,∆AAvolume,∆AAhydropathy,∆AAarea,∆AAweight,∆AAcharge,∆AAflexibily,∆AAchemical,∆AAsize,∆AAhbonds,RSA,Secondary_structure,ddG
1AHW_D178A,91.0,7,13,60,60,85,1,76,7,0.0,2,3,1,0,0,4,13,2,4.0,11,8,0,0,14,60,3,11,13.0,45,10,1,84,60,1,8,45,14.0,12,0,70,85,0,0,10,12,31.0,0,0,1,0,0,1,0,0,0.0,0,76,4,14,84,70,0,0,57.0,428.0,23,71,326,297,266,16,513,23,0.0,7,20,14,8,0,19,71,7,7.0,47,39,17,0,60,326,20,47,85.0,137,68,3,234,297,14,39,137,58.0,53,3,225,266,8,17,68,53,52.0,6,83,16,0,0,3,3,6,0.0,6,513,19,60,234,225,83,6,190.0,482.0,54,186,495,415,209,22,699,54,1.0,6,16,13,18,0,25,

In [19]:
# # Попытка перенаправить вывод в файл для удобства анализа, неудачная

# with open('output.txt', 'w') as f:
#     # Сохраняем оригинальный стандартный вывод
#     original_stdout = sys.stdout
#     # Перенаправляем стандартный вывод в файл
#     sys.stdout = f
#     mCSM = Generate_mCSM(MutationSet, AtomClass, Dmin, Dmax, Dstep)

# sys.stdout = original_stdout

# # Вторая попытка, тоже неудачная, что ж
# %%capture output
# mCSM = Generate_mCSM(MutationSet, AtomClass, Dmin, Dmax, Dstep)

# with open('output_mCSM.txt', 'w') as f:
#     f.write(output.stderr)